In [ ]:
%config IPCompleter.use_jedi = False
%pdb off
# %load_ext viztracer
# from viztracer import VizTracer
%load_ext autoreload
%autoreload 3
import sys
from pathlib import Path

# required to enable non-blocking interaction:
%gui qt5

from copy import deepcopy
from numba import jit
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
# pd.options.mode.dtype_backend = 'pyarrow' # use new pyarrow backend instead of numpy
from attrs import define, field, fields, Factory
import tables as tb
from datetime import datetime, timedelta

# Pho's Formatting Preferences
from pyphocorehelpers.preferences_helpers import set_pho_preferences, set_pho_preferences_concise, set_pho_preferences_verbose
set_pho_preferences_concise()

## Pho's Custom Libraries:
from pyphocorehelpers.Filesystem.path_helpers import find_first_extant_path
from pyphocorehelpers.Filesystem.open_in_system_file_manager import reveal_in_system_file_manager

# Jupyter Widget Interactive
import ipywidgets as widgets
from IPython.display import display
# from pyphoplacecellanalysis.GUI.IPyWidgets.pipeline_ipywidgets import interactive_pipeline_widget, fullwidth_path_widget, interactive_pipeline_files

from pyphocorehelpers.gui.Jupyter.JupyterImageNavigatorWidget import ImageNavigator, ContextSidebar, build_context_images_navigator_widget
from pyphoplacecellanalysis.SpecificResults.PendingNotebookCode import ProgrammaticDisplayFunctionTestingFolderImageLoading
from pyphocorehelpers.indexing_helpers import partition_df_dict, partition
from typing import Dict, List, Tuple, Optional, Callable, Union, Any, NewType
from typing_extensions import TypeAlias
from nptyping import NDArray
import neuropy.utils.type_aliases as types

ContextDescStr = NewType('ContextDescStr', str) # like '2023-07-11_kdiba_gor01_one'
ImageNameStr = NewType('ImageNameStr', str) # like '2006-6-07_11-26-53/kdiba_gor01_one_2006-6-07_11-26-53_maze1__display_1d_placefield_validations.pdf'

import ipywidgets as widgets
from traitlets import HasTraits, observe, Any

# Load Data

In [ ]:


# in_path = Path(r'C:\Users\pho\repos\Spike3DWorkEnv\Spike3D\EXTERNAL\PhoDibaPaper2024Book\data\programmatic_display_function_image_paths.csv').resolve()
in_path: Path = Path('/home/halechr/repos/Spike3D/data/programmatic_display_function_image_paths.csv').resolve()
programmatic_display_function_outputs_df: pd.DataFrame = pd.read_csv(in_path)
sidebar, context_tabs_dict, _final_out_dict_dict = ProgrammaticDisplayFunctionTestingFolderImageLoading.build_ProgrammaticDisplayFunctionTesting_browsing_widget(programmatic_display_function_outputs_df=programmatic_display_function_outputs_df)
sidebar.display()

## OUTPUTS: _final_out_dict_dict

# Main Sessions Image Browsing Widget

In [ ]:
## INPUTS: _final_out_dict_dict: Dict[ContextDescStr, Dict[ImageNameStr, Dict[datetime, Path]]]
context_tabs_dict = {curr_context_desc_str:build_context_images_navigator_widget(curr_context_images_dict, curr_context_desc_str=curr_context_desc_str, max_num_widget_debug=2) for curr_context_desc_str, curr_context_images_dict in list(_final_out_dict_dict.items())}
sidebar = ContextSidebar(context_tabs_dict)
sidebar.display()


In [ ]:
_final_out_dict_dict

# Simple Widget Examples

In [ ]:
import ipywidgets as widgets
from IPython.display import display
from PIL import Image as PILImage
from io import BytesIO
from typing import Dict
from datetime import datetime

class ImageNavigator:
    def __init__(self, image_dict: Dict[datetime, PILImage], image_title: str):
        self.image_dict = image_dict
        self.image_title = image_title
        self.keys = list(image_dict.keys())
        self.current_index = 0

        # Widgets
        self.title_label = widgets.Label(value=self.image_title)
        self.image_display = widgets.Image()
        self.date_label = widgets.Label(value=str(self.keys[self.current_index]))

        self.left_button = widgets.Button(description="←", layout=widgets.Layout(width='50px'))
        self.right_button = widgets.Button(description="→", layout=widgets.Layout(width='50px'))

        # Set up event listeners for the buttons
        self.left_button.on_click(self.on_left_click)
        self.right_button.on_click(self.on_right_click)

        # Initial display setup
        self.update_image()

        # Layout
        self.controls = widgets.HBox([self.left_button, self.date_label, self.right_button])
        self.vbox = widgets.VBox([self.title_label, self.image_display, self.controls])

    def update_image(self):
        """Updates the image display and the date label based on the current index."""
        current_key = self.keys[self.current_index]
        img = self.image_dict[current_key]

        # Convert PIL image to bytes for display in ipywidgets.Image
        with BytesIO() as output:
            img.save(output, format="PNG")
            img_data = output.getvalue()

        # Update the image widget and date label
        self.image_display.value = img_data
        self.date_label.value = str(current_key)

    def on_left_click(self, _):
        """Handle left button click: go to the previous image."""
        self.current_index = (self.current_index - 1) % len(self.keys)
        self.update_image()

    def on_right_click(self, _):
        """Handle right button click: go to the next image."""
        self.current_index = (self.current_index + 1) % len(self.keys)
        self.update_image()

    def display(self):
        """Display the widget."""
        display(self.vbox)

# Example usage with placeholder images
from PIL import Image, ImageDraw

def create_placeholder_image(text):
    img = Image.new('RGB', (200, 200), color='gray')
    d = ImageDraw.Draw(img)
    d.text((10, 90), text, fill=(255, 255, 255))
    return img

image_dict = {
    datetime(2023, 4, 11): create_placeholder_image("2023-04-11"),
    datetime(2023, 5, 12): create_placeholder_image("2023-05-12"),
    datetime(2023, 6, 13): create_placeholder_image("2023-06-13")
}

navigator = ImageNavigator(image_dict, "Image Navigator")
navigator.display()


In [ ]:

# Create some placeholder widgets of varying sizes
widget1 = widgets.Button(description="Widget 1", layout=widgets.Layout(width='150px', height='100px'))
widget2 = widgets.Button(description="Widget 2", layout=widgets.Layout(width='200px', height='150px'))
widget3 = widgets.Button(description="Widget 3", layout=widgets.Layout(width='100px', height='200px'))
widget4 = widgets.Button(description="Widget 4", layout=widgets.Layout(width='250px', height='150px'))
widget5 = widgets.Button(description="Widget 5", layout=widgets.Layout(width='100px', height='100px'))
widget6 = widgets.Button(description="Widget 6", layout=widgets.Layout(width='150px', height='200px'))

# Define a GridBox layout with a varying number of columns
grid = widgets.GridBox(
    [widget1, widget2, widget3, widget4, widget5, widget6],
    layout=widgets.Layout(grid_template_columns="repeat(3, 200px)", grid_gap="10px")
)

# Display the grid
display(grid)

# _pipeline_jupyter_widget = interactive_pipeline_widget(curr_active_pipeline=curr_active_pipeline)
# display(_pipeline_jupyter_widget)
# _pipeline_jupyter_widget

In [ ]:
# <i class="fa-solid fa-clone"></i>
# form_item_layout = ipywidgets.Layout(
#     display='flex',
#     flex_flow='row',
#     justify_content='space-between'
# )

icons = ['clock-rotate-left', 'folder-tree', 'rev', 'file-import', 'file-export', 'upload', 'download', 'file-zipper', 'box-archive', 'message', 'road', 'file-arrow-up', 'file-arrow-down']
rendering_icons = ['vector-square', 'draw-polygon', 'object-group']

button = widgets.Button(
    description='Click me',
    layout=widgets.Layout(flex='0 1 auto', width='auto', margin='0px'),
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
    icon='check' # (FontAwesome names without the `fa-` prefix)
)
secondary_button = widgets.Button(
    description='',
    layout=widgets.Layout(flex='0 1 auto', width='auto', margin='0px'), # means it won't grow beyond its content's width, can shrink if needed, and its basis is auto.
    disabled=False,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
    icon='clone' # (FontAwesome names without the `fa-` prefix)
)
another_button = widgets.Button(description='', layout=widgets.Layout(flex='0 1 auto', width='auto', margin='0px'), disabled=False, button_style='info', tooltip='Click me', icon='clone')

# widgets.HBox([button, secondary_button])

items_auto = [button, secondary_button, another_button]
box_layout = widgets.Layout(display='flex', flex_flow='row', align_items='stretch', width='70%')
box_auto = widgets.Box(children=items_auto, layout=box_layout)
box_auto

In [1]:
import ipywidgets as widgets
import traitlets
from pyphocorehelpers.gui.Jupyter.simple_widgets import CheckBoxListWidget

options_list = ['high_wcorr', 'user_selected', 'high_pearsonr_corr', 'high_shuffle_percentile_score', 'high_shuffle_wcorr_z_score', 'good_jump', 'long_duration']
chk_box_list_widget = CheckBoxListWidget(options_list=options_list)
chk_box_list_widget


Value changed to: ()


CheckBoxListWidget(children=(Checkbox(value=False, description='high_wcorr', layout=Layout(height='auto', marg…

Value changed to: ('high_wcorr',)
Value changed to: ('high_wcorr', 'high_shuffle_percentile_score')
Value changed to: ('high_wcorr', 'high_shuffle_percentile_score', 'long_duration')


In [2]:
chk_box_list_widget.value

()